In [1]:
from nlp_gym.envs.question_answering.env import QAEnv

## 1. Question - Answering Environment
An environment for question answering with multiple choices and supporting facts

**Observation space** :-
Observation consists of triple of Question, Facts, Choice

**Action space** :-
Actions are binary. Either to ANSWER OR CONTINUE
ANSWER corresponds to answering with the choice in the observation
CONTINUE corresponds to agent asking for the next choice

In [2]:
env = QAEnv()

2022-10-14 12:55:15,543 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpl52ybka_


100%|██████████| 1200000128/1200000128 [08:10<00:00, 2446283.99B/s]

2022-10-14 13:03:26,303 copying /tmp/tmpl52ybka_ to cache at /home/nikhil/.flair/embeddings/en-fasttext-news-300d-1M.vectors.npy


2022-10-14 13:03:26,870 removing temp file /tmp/tmpl52ybka_
2022-10-14 13:03:27,189 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-news-300d-1M not found in cache, downloading to /tmp/tmpriia8pk1


100%|██████████| 54600983/54600983 [00:20<00:00, 2703807.65B/s]

2022-10-14 13:03:47,648 copying /tmp/tmpriia8pk1 to cache at /home/nikhil/.flair/embeddings/en-fasttext-news-300d-1M
2022-10-14 13:03:47,699 removing temp file /tmp/tmpriia8pk1


In [4]:
env.get_action_space()

Discrete Action Space with 2 actions: ['ANSWER', 'CONTINUE']

In [5]:
env.get_observation_dim()

2

In [19]:
from nlp_gym.data_pools.custom_question_answering_pools import QASC
from stable_baselines3.dqn import DQN
from stable_baselines3.dqn.policies import DQNPolicy

In [8]:
from stable_baselines3.common.env_checker import check_env

# Prepare the data
train_data = QASC.prepare(split='train')
val_data = QASC.prepare(split='val')

# Add samples to the env
for sample, weight in train_data:
    env.add_sample(sample, weight)

# Check the env
check_env(env, warn=True)

Generating train split:   0%|          | 0/8134 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/920 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/926 [00:00<?, ? examples/s]

Dataset qasc downloaded and prepared to /home/nikhil/.cache/huggingface/datasets/qasc/default/0.1.0/a8c2ff717429f8f9041f665234865cc42c93d4b1b3c4f16a1e119a85366714ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset qasc (/home/nikhil/.cache/huggingface/datasets/qasc/default/0.1.0/a8c2ff717429f8f9041f665234865cc42c93d4b1b3c4f16a1e119a85366714ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
# Train the agent
model = DQN(policy=DQNPolicy, env=env, verbose=0, device='cuda')

In [33]:
model.learn(total_timesteps=1e5)

In [30]:
import tqdm


def eval_model(env_in, model_in, pool):
    correctly_answered = 0.0
    for sam, _ in tqdm.tqdm(pool, desc="Evaluating"):
        obs = env_in.reset(sam)
        state = None
        done = False
        while not done:
            action, state = model_in.predict(obs)
            obs, reward, done, info = env_in.step(int(action))

        if info["selected_choice"] == sam.answer:
            correctly_answered += 1

    return correctly_answered / len(pool)

In [35]:
eval_model(env, model, val_data)

Evaluating: 100%|██████████| 926/926 [00:08<00:00, 107.20it/s]


0.44600431965442766

1e4 timesteps --> 11.7% accuracy
1e5 timesteps --> 44.6% accuracy

In [36]:
eval_model(env, model, train_data)

Evaluating: 100%|██████████| 8134/8134 [01:13<00:00, 109.96it/s]


0.4247602655520039